In [1]:
import numpy as np
import pandas as pd

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')
sample_submission = pd.read_csv('titanic/gender_submission.csv')

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
Xtrain = train.drop(['PassengerId', 'Survived'], axis=1)
Xtest = test.drop(['PassengerId'], axis=1)
Xtrain['is_test'] = False
Xtest['is_test'] = True
X = pd.concat([Xtrain, Xtest], axis=0)
X.index = range(len(X))
X.columns = X.columns.str.lower()

In [7]:
X.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False


In [8]:
X.shape

(1309, 11)

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
pclass      1309 non-null int64
name        1309 non-null object
sex         1309 non-null object
age         1046 non-null float64
sibsp       1309 non-null int64
parch       1309 non-null int64
ticket      1309 non-null object
fare        1308 non-null float64
cabin       295 non-null object
embarked    1307 non-null object
is_test     1309 non-null bool
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 103.6+ KB


__Survived перекодировать не нужно, т.к. она представлена в числовом формате. Просто выделим ее в y.__

In [10]:
y = train.Survived

In [11]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

__Смотрим что есть интересного по фичам.__

In [12]:
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from sklearn import preprocessing

__Готовясь к выполнению домашней работы невозможно не подсмотреть, что в фиче name особое внимание уделяется званию (или обращению) пассажира, поэтому именно это звание (обращение) буду выделять. Остальные слова в имени считаю не интересными.__

__Для выделения звания (обращения) разбиваем строку имени по пробелу. Ищем в полученном массиве строку, которая оканчивается точкой, если таковой нет, оставляем пустую строку. Записываем в новую колонку.__

In [13]:
X['title'] = X['name'].str.split().apply(lambda x: ([s for s in x if s.endswith('.')][0:1] or [''])[0])

In [14]:
X['title'].value_counts()

Mr.          757
Miss.        260
Mrs.         197
Master.       61
Dr.            8
Rev.           8
Col.           4
Mlle.          2
Ms.            2
Major.         2
Countess.      1
Don.           1
Dona.          1
Capt.          1
Sir.           1
Mme.           1
Lady.          1
Jonkheer.      1
Name: title, dtype: int64

In [15]:
X['title'].describe()

count     1309
unique      18
top        Mr.
freq       757
Name: title, dtype: object

In [16]:
len( X[ X['title'] == '' ] )

0

__Посмотрим на важность конкретных званий (обращений)__

In [17]:
one_big_text = " ".join(X.title)
words = one_big_text.split()
most_common = Counter(words).most_common()
most_common[:20]

[('Mr.', 757),
 ('Miss.', 260),
 ('Mrs.', 197),
 ('Master.', 61),
 ('Rev.', 8),
 ('Dr.', 8),
 ('Col.', 4),
 ('Ms.', 2),
 ('Major.', 2),
 ('Mlle.', 2),
 ('Don.', 1),
 ('Mme.', 1),
 ('Lady.', 1),
 ('Sir.', 1),
 ('Capt.', 1),
 ('Countess.', 1),
 ('Jonkheer.', 1),
 ('Dona.', 1)]

In [18]:
Xtitle = pd.DataFrame()
for col, num in most_common:
    Xtitle[col] = X[~X.is_test].title.str.contains(col).astype(int)

In [19]:
Xtitle.head()

,Mr.,Miss.,Mrs.,Master.,Rev.,Dr.,Col.,Ms.,Major.,Mlle.,Don.,Mme.,Lady.,Sir.,Capt.,Countess.,Jonkheer.,Dona.
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
clf = DecisionTreeClassifier(max_depth=4)

In [21]:
clf.fit(Xtitle, train.Survived)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [22]:
clf.feature_importances_

array([ 0.35690301,  0.01020033,  0.58967629,  0.        ,  0.03776846,
        0.        ,  0.        ,  0.        ,  0.        ,  0.00545191,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ])

In [23]:
Xtitle.columns

Index(['Mr.', 'Miss.', 'Mrs.', 'Master.', 'Rev.', 'Dr.', 'Col.', 'Ms.',
       'Major.', 'Mlle.', 'Don.', 'Mme.', 'Lady.', 'Sir.', 'Capt.',
       'Countess.', 'Jonkheer.', 'Dona.'],
      dtype='object')

__Выделим Mr., Miss., Mrs. и Rev.__

In [24]:
X['is_title_mr'] = (X.title == 'Mr.').astype(int)
X['is_title_miss'] = (X.title == 'Miss.').astype(int)
X['is_title_mrs'] = (X.title  == 'Mrs.').astype(int)
X['is_title_rev'] = (X.title == 'Rev.').astype(int)

In [25]:
X.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test,title,is_title_mr,is_title_miss,is_title_mrs,is_title_rev
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,Mr.,1,0,0,0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,Mrs.,0,0,1,0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,Miss.,0,1,0,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,Mrs.,0,0,1,0
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,Mr.,1,0,0,0


__Добавим фичу с количеством слов в имени.__

In [26]:
X['name_words_count'] = X.name.str.split().str.len()

In [27]:
X['sex'].value_counts()

male      843
female    466
Name: sex, dtype: int64

__Преобразуем пол в признак мужского пола.__

In [28]:
X['is_male'] = (X.sex == 'male').astype(int)

In [29]:
X.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test,title,is_title_mr,is_title_miss,is_title_mrs,is_title_rev,name_words_count,is_male
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,Mr.,1,0,0,0,4,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,Mrs.,0,0,1,0,7,0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,Miss.,0,1,0,0,3,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,Mrs.,0,0,1,0,7,0
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,Mr.,1,0,0,0,4,1


In [30]:
X['age'].value_counts()

24.00    47
22.00    43
21.00    41
30.00    40
18.00    39
25.00    34
28.00    32
36.00    31
26.00    30
29.00    30
27.00    30
19.00    29
23.00    26
32.00    24
20.00    23
35.00    23
31.00    23
45.00    21
33.00    21
17.00    20
39.00    20
16.00    19
42.00    18
40.00    18
34.00    16
50.00    15
47.00    14
48.00    14
38.00    14
2.00     12
         ..
28.50     3
12.00     3
0.83      3
34.50     2
71.00     2
45.50     2
70.00     2
14.50     2
30.50     2
0.92      2
36.50     2
23.50     1
67.00     1
60.50     1
38.50     1
0.17      1
55.50     1
0.33      1
20.50     1
24.50     1
66.00     1
0.42      1
70.50     1
11.50     1
26.50     1
76.00     1
22.50     1
74.00     1
80.00     1
0.67      1
Name: age, Length: 98, dtype: int64

In [31]:
X['age'].describe()

count    1046.000000
mean       29.881138
std        14.413493
min         0.170000
25%        21.000000
50%        28.000000
75%        39.000000
max        80.000000
Name: age, dtype: float64

In [32]:
X['age'].isnull().values.any()

True

__В фиче age есть пропуски, нужно заменить их. Заменять на среднее по всем пассажирам не очень хорошая идея, попробую заменить на среднее по полу.__

In [33]:
X['age'] = X.groupby('sex').age.transform(lambda x: x.fillna(x.mean()))

In [34]:
X['age'].describe()

count    1309.000000
mean       29.909496
std        12.889182
min         0.170000
25%        22.000000
50%        30.000000
75%        35.000000
max        80.000000
Name: age, dtype: float64

In [35]:
X['sibsp'].value_counts()

0    891
1    319
2     42
4     22
3     20
8      9
5      6
Name: sibsp, dtype: int64

In [36]:
X['sibsp'].describe()

count    1309.000000
mean        0.498854
std         1.041658
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         8.000000
Name: sibsp, dtype: float64

In [37]:
X['parch'].value_counts()

0    1002
1     170
2     113
3       8
5       6
4       6
9       2
6       2
Name: parch, dtype: int64

In [38]:
X['parch'].describe()

count    1309.000000
mean        0.385027
std         0.865560
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         9.000000
Name: parch, dtype: float64

__Фичи sibsp и parch выглядят полезными (количество родственников может повлиять на выживаемость) и их значения не требуют преобразования.__

In [39]:
X.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test,title,is_title_mr,is_title_miss,is_title_mrs,is_title_rev,name_words_count,is_male
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,Mr.,1,0,0,0,4,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,Mrs.,0,0,1,0,7,0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,Miss.,0,1,0,0,3,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,Mrs.,0,0,1,0,7,0
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,Mr.,1,0,0,0,4,1


__Фичи ticket и cabin не иформативны и буду исключены из обучающей выборки.__

In [40]:
X.groupby('pclass').fare.describe()

,count,mean,std,min,25%,50%,75%,max
pclass,,,,,,,,
1,323.0,87.508992,80.447178,0.0,30.6958,60.0000,107.6625,512.3292
2,277.0,21.179196,13.607122,0.0,13.0000,15.0458,26.0000,73.5000
3,708.0,13.302889,11.494358,0.0,7.7500,8.0500,15.2458,69.5500


In [41]:
X['fare'].describe()

count    1308.000000
mean       33.295479
std        51.758668
min         0.000000
25%         7.895800
50%        14.454200
75%        31.275000
max       512.329200
Name: fare, dtype: float64

In [42]:
len( X[ X['fare'] == 0 ] )

17

In [43]:
len( X[ X['fare'].isnull() ] )

1

__Фича fare имеет 1 пропуск и 17 нулевых значений, но при этом выглядит дублирующей для pclass у которой нет пропусков. Исключаем из модели.__

In [44]:
X['embarked'].value_counts()

S    914
C    270
Q    123
Name: embarked, dtype: int64

In [45]:
X['embarked'].describe()

count     1307
unique       3
top          S
freq       914
Name: embarked, dtype: object

__Фича embarked (порт посадки) имеет пропуски, заменим их на самый популярный порт.__

In [46]:
X['embarked'].fillna(X['embarked'].value_counts().idxmax(), inplace=True)

In [47]:
X['embarked'].describe()

count     1309
unique       3
top          S
freq       916
Name: embarked, dtype: object

In [48]:
X.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test,title,is_title_mr,is_title_miss,is_title_mrs,is_title_rev,name_words_count,is_male
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,Mr.,1,0,0,0,4,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,Mrs.,0,0,1,0,7,0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,Miss.,0,1,0,0,3,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,Mrs.,0,0,1,0,7,0
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,Mr.,1,0,0,0,4,1


__Закодируем фичу embarked.__

In [49]:
le_embarked = preprocessing.LabelEncoder()
X.embarked = le_embarked.fit_transform(X.embarked)

In [50]:
X.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test,title,is_title_mr,is_title_miss,is_title_mrs,is_title_rev,name_words_count,is_male
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,2,False,Mr.,1,0,0,0,4,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,0,False,Mrs.,0,0,1,0,7,0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,False,Miss.,0,1,0,0,3,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,2,False,Mrs.,0,0,1,0,7,0
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,2,False,Mr.,1,0,0,0,4,1


In [51]:
Xtrain_prep = X[~X.is_test].drop(['name', 'sex', 'ticket', 'fare', 'cabin', 'title'], axis=1)
Xtest_prep = X[X.is_test].drop(['name', 'sex', 'ticket', 'fare', 'cabin', 'title'], axis=1)

In [52]:
Xtrain_prep.head()

,pclass,age,sibsp,parch,embarked,is_test,is_title_mr,is_title_miss,is_title_mrs,is_title_rev,name_words_count,is_male
0,3,22.0,1,0,2,False,1,0,0,0,4,1
1,1,38.0,1,0,0,False,0,0,1,0,7,0
2,3,26.0,0,0,2,False,0,1,0,0,3,0
3,1,35.0,1,0,2,False,0,0,1,0,7,0
4,3,35.0,0,0,2,False,1,0,0,0,4,1


In [53]:
Xtest_prep.head()

,pclass,age,sibsp,parch,embarked,is_test,is_title_mr,is_title_miss,is_title_mrs,is_title_rev,name_words_count,is_male
891,3,34.5,0,0,1,True,1,0,0,0,3,1
892,3,47.0,1,0,2,True,0,0,1,0,5,0
893,2,62.0,0,0,1,True,1,0,0,0,4,1
894,3,27.0,0,0,2,True,1,0,0,0,3,1
895,3,22.0,1,1,2,True,0,0,1,0,6,0


In [54]:
Xtrain_prep.shape

(891, 12)

__Приступаем к построению модели.__

In [55]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [56]:
depths = np.arange(1,10)
features_num = np.arange(3,9)
grid = {'max_depth': depths, 'max_features': features_num}
gridsearch = GridSearchCV(DecisionTreeClassifier(), grid, scoring='neg_log_loss', cv=5)

In [57]:
%%time
gridsearch.fit(Xtrain_prep, y)

CPU times: user 1.42 s, sys: 0 ns, total: 1.42 s
Wall time: 1.42 s


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]), 'max_features': array([3, 4, 5, 6, 7, 8])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

In [58]:
sorted(gridsearch.grid_scores_, key = lambda x: -x.mean_validation_score)

/home/myxrome/Applications/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: -0.44080, std: 0.03202, params: {'max_depth': 3, 'max_features': 8},
 mean: -0.45376, std: 0.02439, params: {'max_depth': 2, 'max_features': 7},
 mean: -0.46780, std: 0.02259, params: {'max_depth': 2, 'max_features': 8},
 mean: -0.47193, std: 0.04825, params: {'max_depth': 3, 'max_features': 4},
 mean: -0.48928, std: 0.03474, params: {'max_depth': 2, 'max_features': 3},
 mean: -0.48989, std: 0.01503, params: {'max_depth': 2, 'max_features': 6},
 mean: -0.49400, std: 0.01260, params: {'max_depth': 2, 'max_features': 4},
 mean: -0.49421, std: 0.07924, params: {'max_depth': 3, 'max_features': 5},
 mean: -0.49429, std: 0.06800, params: {'max_depth': 3, 'max_features': 7},
 mean: -0.49932, std: 0.03101, params: {'max_depth': 2, 'max_features': 5},
 mean: -0.51257, std: 0.02620, params: {'max_depth': 1, 'max_features': 5},
 mean: -0.51762, std: 0.02524, params: {'max_depth': 1, 'max_features': 6},
 mean: -0.52880, std: 0.09622, params: {'max_depth': 3, 'max_features': 6},
 mean: -0.55

In [59]:
clf_final = DecisionTreeClassifier(max_depth=3, max_features=8)

In [60]:
clf_final.fit(Xtrain_prep, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [61]:
y_pred_proba = clf_final.predict_proba(Xtest_prep)
y_pred = clf_final.predict(Xtest_prep)

In [62]:
submit = pd.DataFrame(y_pred, columns=sample_submission.columns[1:])
submit['PassengerId'] = sample_submission.PassengerId
submit = submit[[submit.columns[-1]]+list(submit.columns[:-1])]
submit.to_csv('titanic/submit.csv', index=False)

In [63]:
from sklearn.tree import export_graphviz

def get_tree_dot_view(clf, feature_names=None, class_names=None):
    print(export_graphviz(clf, out_file=None, filled=True, feature_names=feature_names, class_names=class_names))

In [64]:
get_tree_dot_view(clf_final, list(Xtrain_prep.columns), ["Not Survived", "Survived"])

digraph Tree {
node [shape=box, style="filled", color="black"] ;
0 [label="is_male <= 0.5\ngini = 0.473\nsamples = 891\nvalue = [549, 342]\nclass = Not Survived", fillcolor="#e5813960"] ;
1 [label="pclass <= 2.5\ngini = 0.383\nsamples = 314\nvalue = [81, 233]\nclass = Survived", fillcolor="#399de5a6"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="age <= 2.5\ngini = 0.1\nsamples = 170\nvalue = [9, 161]\nclass = Survived", fillcolor="#399de5f1"] ;
1 -> 2 ;
3 [label="gini = 0.5\nsamples = 2\nvalue = [1, 1]\nclass = Not Survived", fillcolor="#e5813900"] ;
2 -> 3 ;
4 [label="gini = 0.091\nsamples = 168\nvalue = [8, 160]\nclass = Survived", fillcolor="#399de5f2"] ;
2 -> 4 ;
5 [label="embarked <= 1.5\ngini = 0.5\nsamples = 144\nvalue = [72, 72]\nclass = Not Survived", fillcolor="#e5813900"] ;
1 -> 5 ;
6 [label="gini = 0.423\nsamples = 56\nvalue = [17, 39]\nclass = Survived", fillcolor="#399de590"] ;
5 -> 6 ;
7 [label="gini = 0.469\nsamples = 88\nvalue = [55, 33]\nc